In [ ]:
from nilearn.plotting import plot_anat


pre_lesion = '/home/ajoshi/Desktop/lesion_sim/Subject_23/Subject_23/core_lesion_mask.nii.gz'
post_lesion = '/home/ajoshi/Desktop/lesion_sim/Subject_23/Subject_23/lesion_mask.nii.gz'
mri = '/home/ajoshi/Desktop/lesion_sim/Subject_23/Subject_23/original_image.nii.gz'

pre2post_lesion = '/home/ajoshi/Desktop/lesion_sim/Subject_23/Subject_23/output_pre2post_lesion.nii.gz'
ddf = '/home/ajoshi/Desktop/lesion_sim/Subject_23/Subject_23/output_pre2post_lesion_ddf.nii.gz'
jac_file = '/home/ajoshi/Desktop/lesion_sim/Subject_23/Subject_23/output_pre2post_lesion_jac.nii.gz'
output_file_pre2post_lesion='/home/ajoshi/Desktop/lesion_sim/Subject_23/Subject_23/pre2post_lesion_check.nii.gz'
output_file_pre2post_mri='/deneb_disk/auto_resection/lesion_masks/pre2post_lesion.nii.gz'


cross_hair = (61.27,-31,11.13) #(56, 105, 122)
plot_anat(pre_lesion, title='Pre-lesion mask',cut_coords=cross_hair)
plot_anat(post_lesion, title='Post-lesion mask', cut_coords=cross_hair)


In [ ]:

from warper import Warper

nonlin_reg = Warper()

nonlin_reg.nonlinear_reg(
    target_file=post_lesion,
    moving_file=pre_lesion,
    output_file=pre2post_lesion,
    ddf_file=ddf,
    reg_penalty=3,
    nn_input_size=64,
    lr=1e-3,
    max_epochs=3000,
    loss="mse",
    jacobian_determinant_file=jac_file)


In [ ]:
from warp_utils import apply_warp
from monai.transforms import LoadImage, EnsureChannelFirst
import nibabel as nib

plot_anat(pre_lesion, title='pre-lesion', cut_coords=cross_hair)
plot_anat(post_lesion, title='post-lesion', cut_coords=cross_hair)
plot_anat(pre2post_lesion, title='pre2post-lesion', cut_coords=cross_hair)


moving = LoadImage(image_only=True)(pre_lesion)
moving = EnsureChannelFirst()(moving)[None]
target = LoadImage(image_only=True)(post_lesion)
target = EnsureChannelFirst()(target)[None]
disp = LoadImage(image_only=True)(ddf)
disp = EnsureChannelFirst()(disp)[None]
mri = LoadImage(image_only=True)(mri)
mri = EnsureChannelFirst()(mri)[None]

moved = apply_warp(disp_field=disp, moving_image=moving, target_image=target)
nib.save(nib.Nifti1Image(moved[0, 0].detach().cpu().numpy(), target.affine), output_file_pre2post_lesion)


target = LoadImage(image_only=True)(post_lesion)
target = EnsureChannelFirst()(target)[None]
disp = LoadImage(image_only=True)(ddf)
disp = EnsureChannelFirst()(disp)[None]
#mri = LoadImage(image_only=True)(mri)
#mri = EnsureChannelFirst()(mri)[None]
output_file='/deneb_disk/auto_resection/lesion_masks/pre2post_mri.nii.gz'
moved = apply_warp(disp_field=disp, moving_image=mri, target_image=target)
nib.save(nib.Nifti1Image(moved[0, 0].detach().cpu().numpy(), target.affine), output_file)


In [ ]:

lesion_file = '/deneb_disk/auto_resection/lesion_masks/lesion_init0.mask.nii.gz'
lesion_mri = '/deneb_disk/auto_resection/lesion_masks/USCBrain.pre_lesion.nii.gz'

target = LoadImage(image_only=True)(post_lesion)
target = EnsureChannelFirst()(target)[None]
disp = LoadImage(image_only=True)(ddf)
disp = EnsureChannelFirst()(disp)[None]
lesion = LoadImage(image_only=True)(lesion_file)
lesion = EnsureChannelFirst()(lesion)[None]
lesion = mri + lesion
nib.save(nib.Nifti1Image(lesion[0, 0].detach().cpu().numpy(), target.affine), lesion_mri)

moved = apply_warp(disp_field=disp, moving_image=lesion, target_image=target)
nib.save(nib.Nifti1Image(moved[0, 0].detach().cpu().numpy(), target.affine), output_file_pre2post_mri)


plot_anat(lesion_mri, title='Pre lesion mask', cut_coords=cross_hair)
plot_anat(output_file, title='Pre2post lesion mask', cut_coords=cross_hair)
